In [1]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162999 sha256=5c7fd191c62b419153053423e1fb1328d3d2dbd312dbab3cf84c6c5b4142bf24
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [4]:
import pandas as pd
import numpy as np
import surprise

In [5]:
ratings = pd.read_csv("ratings.txt",sep=' ',names = ['uid','iid','rating'])
ratings.head()

,uid,iid,rating
0,1,1,2.0
1,1,2,4.0
2,1,3,3.5
3,1,4,3.0
4,1,5,4.0


In [6]:
lowest_rating = ratings['rating'].min()
highest_rating = ratings['rating'].max()
print("Ratings range between {0} and {1}".format(lowest_rating,highest_rating))
reader = surprise.Reader(rating_scale = (lowest_rating,highest_rating))

Ratings range between 0.5 and 4.0


Converting the data into **surprise** format:

In [7]:
data = surprise.Dataset.load_from_df(ratings,reader)
type(data)

surprise.dataset.DatasetAutoFolds

**Similarity options:**

In case of Item based filtering, `user_based` value will be `False`

In [8]:
similarity_options = {'name': 'cosine', 'user_based': False}

Default K = 40

In [9]:
algo = surprise.KNNBasic(sim_options = similarity_options)
output = algo.fit(data.build_full_trainset())

Computing the cosine similarity matrix...
Done computing similarity matrix.


The above `.fit()` calculates expected rating for all the users

Say we want expected rating of user=50 for item 217

In [10]:
pred = algo.predict(uid='50',iid='217')
pred

Prediction(uid='50', iid='217', r_ui=None, est=3.0028030537791928, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})

In [11]:
score = pred.est
print(score)


3.0028030537791928


All the users

In [12]:
iids = ratings['iid'].unique()
iids

array([   1,    2,    3, ..., 2069, 2070, 2071])

In [13]:
len(iids)

2071

Consider ratings given by user=50

In [14]:
user_no=50

In [15]:
rec_u = ratings[ratings['uid'] == user_no ]
iids_u = rec_u['iid']
print("List of iid that uid={0} has rated:".format(user_no))
print(iids_u.values)

List of iid that uid=50 has rated:
[  8 211   3   2 219 234  12 254 250 207  11 253 236  84  10   7 233  13
   1   5   6 252 241 216 257 206   4 217   9 215 213  17 255 220 121 245
 239 251 235]


In [16]:
iids_to_predict = np.setdiff1d(iids,iids_u)
print("List of iid which uid={0} did not rate(in all {1}) :".format(user_no,len(iids_to_predict)))
print(iids_to_predict)

List of iid which uid=50 did not rate(in all 2032) :
[  14   15   16 ... 2069 2070 2071]


In [17]:
len(iids_to_predict)

2032

Create a testset for getting the expected rating

In [18]:
testset = [[user_no,iid,0.] for iid in iids_to_predict]
testset

[[50, 14, 0.0],
 [50, 15, 0.0],
 [50, 16, 0.0],
 [50, 18, 0.0],
 [50, 19, 0.0],
 [50, 20, 0.0],
 [50, 21, 0.0],
 [50, 22, 0.0],
 [50, 23, 0.0],
 [50, 24, 0.0],
 [50, 25, 0.0],
 [50, 26, 0.0],
 [50, 27, 0.0],
 [50, 28, 0.0],
 [50, 29, 0.0],
 [50, 30, 0.0],
 [50, 31, 0.0],
 [50, 32, 0.0],
 [50, 33, 0.0],
 [50, 34, 0.0],
 [50, 35, 0.0],
 [50, 36, 0.0],
 [50, 37, 0.0],
 [50, 38, 0.0],
 [50, 39, 0.0],
 [50, 40, 0.0],
 [50, 41, 0.0],
 [50, 42, 0.0],
 [50, 43, 0.0],
 [50, 44, 0.0],
 [50, 45, 0.0],
 [50, 46, 0.0],
 [50, 47, 0.0],
 [50, 48, 0.0],
 [50, 49, 0.0],
 [50, 50, 0.0],
 [50, 51, 0.0],
 [50, 52, 0.0],
 [50, 53, 0.0],
 [50, 54, 0.0],
 [50, 55, 0.0],
 [50, 56, 0.0],
 [50, 57, 0.0],
 [50, 58, 0.0],
 [50, 59, 0.0],
 [50, 60, 0.0],
 [50, 61, 0.0],
 [50, 62, 0.0],
 [50, 63, 0.0],
 [50, 64, 0.0],
 [50, 65, 0.0],
 [50, 66, 0.0],
 [50, 67, 0.0],
 [50, 68, 0.0],
 [50, 69, 0.0],
 [50, 70, 0.0],
 [50, 71, 0.0],
 [50, 72, 0.0],
 [50, 73, 0.0],
 [50, 74, 0.0],
 [50, 75, 0.0],
 [50, 76, 0.0],
 [50, 77

Generate predictions on testset

In [19]:
predictions = algo.test(testset)
predictions[1000]

Prediction(uid=50, iid=1040, r_ui=0.0, est=3.540093440208082, details={'actual_k': 38, 'was_impossible': False})

In [20]:
type(predictions)

list

In [21]:
pred_ratings = np.array([pred.est for pred in predictions])
pred_ratings

array([3.60714286, 3.52692492, 3.56540385, ..., 3.75      , 3.75      ,
       3.46875   ])

In [22]:
iids_to_predict

array([  14,   15,   16, ..., 2069, 2070, 2071])

In [23]:
df_exp_ratings = pd.DataFrame({'iid':iids_to_predict, 'exp_rating':pred_ratings})
df_exp_ratings.head()

,iid,exp_rating
0,14,3.607143
1,15,3.526925
2,16,3.565404
3,18,3.607143
4,19,3.607143


In [24]:
df_exp_ratings.sort_values('exp_rating', ascending=False)

,iid,exp_rating
218,258,4.000000
1805,1845,4.000000
1766,1806,4.000000
1167,1207,3.833333
1165,1205,3.785714
...,...,...
1826,1866,3.000000
1737,1777,3.000000
1736,1776,3.000000
1169,1209,2.500000


Top 10 Items' Expected ratings

In [25]:
df_exp_ratings.sort_values('exp_rating', ascending=False).head(10)

,iid,exp_rating
218,258,4.000000
1805,1845,4.000000
1766,1806,4.000000
1167,1207,3.833333
1165,1205,3.785714
2028,2068,3.750000
896,936,3.750000
1689,1729,3.750000
2030,2070,3.750000
2029,2069,3.750000


Tuning with different K

In [44]:
from surprise.model_selection import GridSearchCV
from surprise.model_selection.split import KFold

param_grid = {'k': np.arange(20,110,10),
              'sim_options': {'name': ['cosine'],
                              'user_based': [False]}}
kfold = KFold(n_splits=5, random_state=24, shuffle=True)
gs = GridSearchCV(surprise.KNNBasic, param_grid, measures=['rmse', 'mae'], cv=kfold)

Running the Grid Search CV

In [45]:
gs.fit(data)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing th

**Best Score**

In [48]:
print(gs.best_score['rmse'])

0.8250322199104293


**Best Param**

In [49]:
print(gs.best_params['rmse'])

{'k': 20, 'sim_options': {'name': 'cosine', 'user_based': False}}


Tuning with different K, similarities and Types

In [56]:
from surprise.model_selection import GridSearchCV
from surprise.model_selection.split import KFold

param_grid = {'k': np.arange(20,90,10),
              'sim_options': {'name': ['cosine','pearson_baseline','MSD'],
                              'user_based': [False, True]}}
kfold = KFold(n_splits=5, random_state=24, shuffle=True)
gs = GridSearchCV(surprise.KNNBasic, param_grid, measures=['rmse', 'mae'], cv=kfold)

Running the Grid Search CV

In [57]:
gs.fit(data)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estima

**Best Score**

In [60]:
print(gs.best_score['rmse'])

0.8177583813567078


**Best Param**

In [61]:
print(gs.best_params['rmse'])

{'k': 20, 'sim_options': {'name': 'MSD', 'user_based': False}}


`KNNWithZScore` considering the normalized z-scores for each user

In [62]:
param_grid = {'k': np.arange(20,90,10),
              'sim_options': {'name': ['cosine','pearson_baseline','MSD'],
                              'user_based': [False, True]}}
kfold = KFold(n_splits=5, random_state=24, shuffle=True)
gs = GridSearchCV(surprise.KNNWithZScore, param_grid, measures=['rmse', 'mae'], cv=kfold)

In [63]:
gs.fit(data)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estima

**Best Score**

In [64]:
print(gs.best_score['rmse'])

0.8174332959162086


**Best Param**

In [65]:
print(gs.best_params['rmse'])

{'k': 70, 'sim_options': {'name': 'pearson_baseline', 'user_based': True}}
